<a href="https://colab.research.google.com/github/tluxxx/PortfolioExperiments/blob/main/GebertIndicatorTest(1993_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gebert Indicator

#1. General Preparations

In [5]:
!pip install ecbdata

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
# import data processing libraries
import pandas as pd
import numpy as np
import datetime as dt

# import plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import data providing libraries
import yfinance as yf
import pandas_datareader as pdr
from ecbdata import ecbdata

#2. Downloading required data and data preparations

## 2.1 DAX-data

In [3]:
# importing the DAX-data
start_period, end_period = '1992-01-01', '2024-05-31'   # download data for analyses
start_assessment = '1993-01-01'                         # start of Indicator assessment

ticker ='^GDAXI'
dax_d = yf.download(ticker, start=start_period, end=end_period)

# Resampling to identify the first trading day of the month and the corresponding prices
dax_d['date'] = pd.to_datetime(dax_d.index).date
trade_dates_m = dax_d.resample('MS').first()['date'].to_list()
dax_m = dax_d[dax_d.index.isin(trade_dates_m)]

[*********************100%%**********************]  1 of 1 completed


## 2.2 Inflation Data

In [7]:
# reading and preparing inflation data for 1991-2024
# data from German Statistical Office (1991-1998)
data_ger = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/GermanInflationData(1991-2024).xlsx', sheet_name='DATA')
inflation_ger = pd.Series(data_ger['chg_12m'].tolist(), index=data_ger['date'])


# download data from ECB (1197-2024)
data_key = 'ICP.M.U2.N.000000.4.ANR'  # HICP - Overall index, Euro area (changing composition), Monthly
inflation_eu = ecbdata.get_series(data_key, start=start_period, end=end_period)
inflation_eu.TIME_PERIOD = pd.to_datetime(inflation_eu.TIME_PERIOD)
inflation_eu = inflation_eu.set_index('TIME_PERIOD')
inflation_eu = inflation_eu.OBS_VALUE.astype(float)

inflation_ger = inflation_ger.loc[start_period:]
inflation_ger.loc[start_period] = np.nan
cut_off_date ='1998-01-31'
df1 = inflation_ger.loc[start_period:cut_off_date]
df2 = inflation_eu.loc[cut_off_date:]
inflation = pd.concat([df1,df2])
inflation.name = 'inflation'

## 2.3. Interest Rates

In [19]:
# reading and preparing interest data from Central Bank Sources

# German Bundesbank discount rates for every day (approximation, all values as of 01-of each month)
data_db = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/DiscountRatesGermanBundesbank (1990-1998).xlsx',sheet_name='DATA' )
data_db['date'] = pd.to_datetime(data_db['date'])

# generation of data frame with relevant data
## monthly data
end_date_int = '1998-12-31'
disc_x = (data_db[data_db['date'] >= start_period]['interest'] / 100).tolist()
disc_r = pd.DataFrame({'date':data_db[data_db['date'] >= start_period]['date'].tolist(),
                       'interest':disc_x})
## conversion to daily data (approximation: if interest rate changes during a month, the change will be implemented at first day of following month
dates = pd.date_range(start_period, end_date_int)
values = np.nan * len(dates)
dates_d = pd.DataFrame({'date': dates,
                        'interest': values})
xx = pd.merge(dates_d, disc_r, how='left', on='date')['interest_y']
xx.ffill(inplace=True)
db_int = pd.DataFrame({'interest':xx.tolist()},
                      index = dates)
db_int['interest'] = db_int['interest'] * 100

# ECB interest data for every day
# download data
data_key = 'FM.D.U2.EUR.4F.KR.MRR_RT.LEV'     # Main refinancing operations - Minimum bid rate/fixed rate (date of changes) - Level, Euro area (changing composition), Daily
ecb_int = ecbdata.get_series(data_key, start=start_period, end=end_period)
ecb_int.TIME_PERIOD = pd.to_datetime(ecb_int.TIME_PERIOD)
ecb_int = ecb_int.set_index('TIME_PERIOD')
ecb_int = ecb_int.OBS_VALUE.astype(float)

# aggregation of German Discount rate with ECB key interest rate
df2 = pd.DataFrame({'interest':ecb_int.tolist()}, index=ecb_int.index)
x = pd.concat([db_int, df2])

interest = pd.concat([db_int, x])
interest = interest['interest'].astype(float)
int_agg = interest[~interest.index.duplicated(keep='first')]

## 2.3 Exchange rates EURO - USD

In [22]:
# Exchange rates
switch_date = '1999-01-01'   # switch from German Bundesbank to ECB data

# importing the USD-DM historical exchange rates
data_db = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/USDDM_HistoricalRatesAdaptedVersion_1960_1998.xlsx', index_col='Date')
# type conversion and supplementing missing dates (ffill)
data_db['rate'] = data_db['rate'].str.replace('.', '0')
data_db['rate'] = data_db['rate'].str.replace(',', '.')
data_db['rate'] = data_db['rate'].astype(float)
data_db['rate'].replace(0, np.nan, inplace=True)
data_db['rate'].ffill(inplace=True)

usddm_db = data_db.loc[start_period:]           # USD-DM data
eurusd_db = 1 / usddm_db.div(1.95583)           # reverting the currency pair and transforming to EURO
eurusd_db.index.names = ['DATE']
eurusd_db.index = pd.to_datetime(eurusd_db.index)

# importing the EURO-USD historical exchange rates
eurusd_fred = pdr.data.DataReader('DEXUSEU', 'fred', switch_date, end_period)
eurusd_fred['DEXUSEU'].ffill(inplace=True)
eurusd_fred.rename(columns = {'DEXUSEU':'rate'}, inplace = True)

# combining Bundesbank and FRED data to one timeseries
df1 = eurusd_db
df2 = eurusd_fred
eurusd = pd.concat([df1,df2])
eurusd = eurusd['rate']
eurusd.name = 'eurusd'

# 3. Calculation of the Gebert-Indicator and Trade Positions

In [24]:
# collecting raw data for Gebert/Indicator-Calculation

# generation empty dataframe for raw-data with all first callendar days of a month as index
dates_param = pd.date_range(start_period, end_period, freq ='MS')
params = pd.DataFrame(index=dates_param)

# collecting the first trading days of a month
params['trade_days'] = trade_dates_m

# collection of monthly dax-data
x = dax_m.Close.tolist()
params['dax'] = x

# collection of interest data and adding to dataframe
intr = int_agg[int_agg.index.isin(dates_param)]
params = params.join(intr)

# collection of inflation data and adding to dataframe
infl = inflation[inflation.index.isin(dates_param)]
params = params.join(infl)

# collection of exchange rate data
currency = eurusd[eurusd.index.isin(trade_dates_m)]
params['eurusd'] = currency.tolist()

# collection input for saisonality
params['seasonality'] = params.index.month


In [26]:
# calculation of subindicators, the resulting Gebert-Indicator and Positions

# subindicator interest: +1: if last adaptation of CB's benchmark interest rate was a reduction, 0: otherwise
interest_chg = params['interest'].diff()
params['ind_int_chg'] = np.select([interest_chg < 0, interest_chg > 0, interest_chg == 0], [1, 0, np.nan])
params['ind_int_chg'].ffill(inplace=True)

# subindicator inflation: +1: if inflation rate declined, compared to -12m value, 0: otherwise
params['ind_infl_chg'] = np.where(params['inflation'] < params['inflation'].shift(12), 1, 0)

# subindicator exchange rates USD/EUR: +1: if USD got stronger compared to -12m, 0: otherwise
params['ind_eurusd_chg'] = np.where(params['eurusd'] < params['eurusd'].shift(12), 1, 0)

# subindicator seasonality: +1: during [November -April], 0: otherwise
params['ind_seasonality'] = np.where(params['seasonality'].isin([11,12,1,2,3,4]), 1, 0)

# Gebert-Indicator and Trade postions
params['GI'] = params['ind_int_chg'] + params['ind_infl_chg'] + params['ind_eurusd_chg'] + params['ind_seasonality']

# calculation of trade positons: GI=3,4 stay/go LONG; GI=0,1 stay/go FLAT, GI=2 keep positions
conditions = [params['GI'] <= 1, params['GI'] >= 3, params['GI'] == 2]
choices = [0, 1, np.nan]
params['positions'] = np.select(conditions, choices)
params['positions'].ffill(inplace=True)


In [39]:
# profitability Gebert indicator system - accumulated returns
params_calc = params.loc[start_assessment:]
params_calc['pnl'] = (1 + params_calc['dax'].pct_change() * params_calc['positions']).cumprod()

# Buy and Hold data as reference
params_calc['pnl_buh'] = (1 + params_calc['dax'].pct_change()).cumprod()


In [40]:
# visualisation of PnL
main_title = 'development of PnL position of Gebert-Indicator-Strategy vs. B&H'
sub_title = f'instrument: {ticker}, analyzed  from: {start_assessment} to:{end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = make_subplots(rows=3, cols=1, row_heights=[0.8,0.1,0.1], shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=params_calc.index, y=params_calc['pnl'], name = 'Gebert Strategy - accumulated returns'), row=1, col=1)
fig.add_trace(go.Scatter(x=params_calc.index, y=params_calc['pnl_buh'], name = 'Buy and Hold'), row=1, col=1)
fig.add_trace(go.Scatter(x=params_calc.index, y=params_calc['GI'], line_shape='hv', name = 'Gebert-Indicator-Value'), row=2, col=1)
fig.add_trace(go.Scatter(x=params_calc.index, y=params_calc['positions'], line_shape='hv', line_color='green', fill='tozeroy', name='position'), row=3, col=1)
fig.update_layout(title=title, template='plotly_dark', autosize=False, width=1500, height=1000)
fig.update_yaxes(title_text='PnL', row=1, col=1)
fig.update_yaxes(title_text='GI', row=2, col=1)
fig.update_yaxes(title_text='position', row=3, col=1)
fig.update_xaxes(title_text='time', row=3, col=1)
fig.show()

In [41]:
# Drawdown
x0 = ((params_calc['pnl'] / params_calc['pnl'].expanding().max()) - 1) * 100
x1 = ((params_calc['pnl_buh'] / params_calc['pnl_buh'].expanding().max()) - 1) * 100

main_title = 'drawdown Gebert Strategy vs. Buy & Hold'
sub_title = f'instrument: {ticker}   from   {start_assessment}   to   {end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=x0.index, y=x0, mode='none', fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.3)', name='Gebert-Strategy'))
fig.add_trace(go.Scatter(x=x1.index, y=x1, mode='none', fill='tozeroy', fillcolor='rgba(255, 255, 0, 0.3)', name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark',autosize=False, width=1500, height=400)
fig.update_layout(title=title, xaxis_title='date', yaxis_title='drawdown (%)')
fig.show()



In [42]:
# Drawdown - absolute
x0 = params_calc['pnl'] - params_calc['pnl'].expanding().max()
x1 = params_calc['pnl_buh'] - params_calc['pnl_buh'].expanding().max()

main_title = 'drawdown Gebert Strategy vs. Buy & Hold'
sub_title = f'instrument: {ticker}   from   {start_assessment}   to   {end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=x0.index, y=x0, mode='none', fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.3)', name='Gebert-Strategy'))
fig.add_trace(go.Scatter(x=x1.index, y=x1, mode='none', fill='tozeroy', fillcolor='rgba(255, 255, 0, 0.3)', name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark',autosize=False, width=1500, height=400)
fig.update_layout(title=title, xaxis_title='date', yaxis_title='drawdown (abs)')
fig.show()

In [43]:
# behind the scenes
main_title = 'development of PnL position of Gebert-Indicator-Strategy vs. B&H'
sub_title = f'instrument: {ticker}, analyzed  from: {start_assessment} to:{end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig=go.Figure()
fig.add_trace(go.Scatter(x=params_calc.index, y=params_calc['pnl'], name = 'Gebert Strategy - accumulated returns'))
fig.add_trace(go.Scatter(x=params_calc.index, y=params_calc['pnl_buh'], name = 'Buy and Hold'))
fig.update_yaxes(type='log')
fig.update_layout(template='plotly_dark', autosize=False, width=1500, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='log PnL', legend_title='Sources')
fig.show()


In [70]:
# simulation of the difference to B&H at various entry dates

start_verification = '1993-02-01'
end_verification = '2019-12-31'

test_starts = pd.date_range(start=start_verification, end=end_verification, freq='MS')
results_strat, results_buh = [], []

for start in test_starts:
  params_test = params.loc[start:]
  params_test['pnl'] = 0 * len(params_test)
  params_test['buh'] = 0 * len(params_test)
  params_test['pnl'] = (1 + params_test['dax'].pct_change() * params_test['positions']).cumprod()
  params_test['buh'] = (1 + params_test['dax'].pct_change()).cumprod()
  results_strat.append(params_test['pnl'].iloc[-1])
  results_buh.append(params_test['buh'].iloc[-1])

results = pd.DataFrame({'pnl_strat':results_strat,
                        'pnl_buh': results_buh},
                       index= test_starts)


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
<ipython-input-70-dfa15f922aa2>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-70-dfa15f922aa2>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-70-dfa15f922aa2>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [71]:
# behind the scenes
main_title = 'development of PnL position of Gebert-Indicator-Strategy vs. B&H'
sub_title = f'instrument: {ticker}, analyzed  from various start-dates to:{end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig=go.Figure()
fig.add_trace(go.Scatter(x=results.index, y=results['pnl_strat'], name = 'Gebert Strategy - accumulated returns'))
fig.add_trace(go.Scatter(x=results.index, y=results['pnl_buh'], name = 'Buy and Hold'))

fig.update_layout(template='plotly_dark', autosize=False, width=1500, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='final PnL', legend_title='Parameters')
fig.show()

In [72]:
x1 = params_calc.loc['2009-03-01']['dax']
x2 = params_calc.iloc[-1]['dax']

print(f' Dax_end/Dax_start (start = 2009-03-01):{x2/x1:.3f}     Dax_start: {x1:.2f}    Dax_end: {x2:.2f} ')


 Dax_end/Dax_start (start = 2009-03-01):4.824     Dax_start: 3710.07    Dax_end: 17896.50 


In [73]:
# behind the scenes
main_title = 'development of PnL position of Gebert-Indicator-Strategy vs. B&H'
sub_title = f'instrument: {ticker}, analyzed  from: {end_verfication} to:{end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig=go.Figure()
fig.add_trace(go.Scatter(x=params_test.index, y=params_test['pnl'], name = 'Gebert Strategy - accumulated returns'))
fig.add_trace(go.Scatter(x=params_test.index, y=params_test['buh'], name = 'Buy and Hold'))

fig.update_layout(template='plotly_dark', autosize=False, width=1500, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='PnL', legend_title='Parameters')
fig.show()